In [1]:
import helpers
import numpy as np
import re
import math
from tqdm import tqdm
import copy
import networkx as nx
from collections import Counter
import ast
import itertools as it

data = """
..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..###..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#..#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#......#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#.....####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.......##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

#..#.
#....
##..#
..#..
..###
"""

with open("input_20.txt", 'r') as f:
    data = f.read()
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = re.findall("(\d+),(\d+) -> (\d+),(\d+)", data)
#data = [x.rstrip() for x in data.split("\n\n") if x]

translation = {"#": 1, ".": 0}

data = data.lstrip()
data = data.rstrip()

decoder, array = data.split("\n\n")
decoder = [int(translation[x]) for x in decoder]

array = np.array([np.array([int(translation[y]) for y in x]) for x in array.split("\n")])
print(decoder)
print(array)


[1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 

In [2]:
from scipy.spatial import Delaunay
def manhattan(a, b):
    return sum(abs(val1-val2) for val1, val2 in zip(a,b))

In [3]:
def getNeighboursInRow(array, pos, outside_area = "0", way=9):
    if way == 4:
        DYDX = [(1,0),(0,1), (-1,0), (0,-1)]
    elif way == 8:
        DYDX = [(1,0),(0,1), (-1,0), (0,-1), (1,1),(-1,1), (-1,-1), (1,-1)]
    elif way == 9:
        DYDX = [(1,0),(0,1), (-1,0), (0,-1), (1,1),(-1,1), (-1,-1), (1,-1)]
        DYDX = [(-1,-1), (0,-1), (1,-1), (-1,0), (0,0), (1,0), (-1,1), (0,1), (1,1)]
    else:
        print("inproper way")
        return
    n_list = []
    pos_list = []
    answer = ""
    for dydx in DYDX:
        new_pos = (pos[0]+dydx[0],pos[1]+dydx[1])
        if -1 < new_pos[0] < len(array[0]) and -1 < new_pos[1] < len(array):
            answer += str(array[new_pos[1]][new_pos[0]])
            pos_list.append((new_pos[0],new_pos[1]))
        else:
            answer += outside_area

    return int(answer, 2)

In [4]:
oa ={"1": "0", "0": "1"}
outsideArea = "0"
for x in tqdm(range(50)):
    array = np.pad(array, [(3, ), (3, )], mode='constant', constant_values=int(outsideArea))
    new_array = np.zeros_like(array)
    for y, r in enumerate(array):
        for x, val in enumerate(r):
            new_array[y,x] = decoder[getNeighboursInRow(array, (x,y), outsideArea)]
    outsideArea = oa[outsideArea]
    array = new_array

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:45<00:00,  1.10it/s]


In [5]:
#helpers.npNicePrint(array)
print(sum(sum(array)))

16112
